In [1]:
!pip install SimPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random

import simpy
condition_flag = None
countt=0
RANDOM_SEED = 42
NEW_CUSTOMERS = 20  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
def source(env, number, interval, counter):

    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
    return countt


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    global countt
    global condition_flag
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))
    countt+=1
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            if countt>5:
              for i in range(20):
                counter = simpy.Resource(env, capacity=i+1)
              print('%7.4f %s:counter opened %6.3f' % (env.now, counter,wait))            
            #print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
#print('%7.4f %s: Here I am' % (arrive, name))
#print(countt)
# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
#countt=source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)

counter = simpy.Resource(env, capacity=1)

env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished
47.5190 Customer05: Here I am
47.5190 Customer05: Waited  0.000
50.5073 Customer05: Finished
58.0139 Customer06: Here I am
58.0139 Customer06: Waited  0.000
58.0921 Customer06: Finished
66.9120 Customer07: Here I am
66.9120 Customer07: Waited  0.000
71.9027 Customer07: Finished
83.3016 Customer08: Here I am
83.3016 Customer08: Waited  0.000
84.9915 Customer09: Here I am
85.9648 Customer10: Here I am
86.1849 <simpy.resources.resource.Resource object at 0x7fbe380eb760>:counter opened  1.193
88.1723 <simpy.resources.resource.Resource object at 0x7fbe380eb100>:counter opened  2.207
8